In [ ]:
"""
公開資訊觀測站
https://mops.twse.com.tw/mops/web/index
雖然服務已經公告停用，但還是可以透過網址查詢新的數據如下
https://mops.twse.com.tw/nas/t21/sii/t21sc03_111_1.html
"""
# 載入必要的套件
import time
import requests
from bs4 import BeautifulSoup
import os

# import datetime
import random

# from json import loads
import pandas as pd

# 添加 StringIO 取代 pd.read_html
from io import StringIO

# import sys

# 切換工作路徑
os.chdir("./")

ym_list = []
for y in range(2022, 2023):
    for m in range(1, 13):
        ym_list.append([str(y), str(m).zfill(2)])

# 從 2010年開始爬蟲
first_write = True

for y, m in ym_list:
    # ym = y + m
    ym = f"{y}{m}"
    try:
        print(ym)

        y = int(y) - 1911
        m = int(m)

        url = f"https://mops.twse.com.tw/nas/t21/sii/t21sc03_{y}_{m}.html"
        # 下載該年月的網站，並用 pandas 轉換成 dataframe
        r = requests.get(url)
        # 這個連中的資料使用 big5 編碼
        r.encoding = "big5"
        # 添加 bs
        soup = BeautifulSoup(r.text, "html.parser")
        # 使用 BeautifulSoup 找到所有的表格
        tables = soup.find_all("table")
        # 處理每個表格
        dfs = []
        for table in tables:
            # 將 HTML 表格轉換為 DataFrame
            df = pd.read_html(StringIO(str(table)), encoding="big5")[0]
            if df.shape[1] <= 11 and df.shape[1] > 5:
                dfs.append(df)

        # 這裡可以進行進一步的數據處理和儲存
        data = pd.concat(dfs)

        # 調整欄位名稱
        if data.shape[1] == 10:
            data.columns = [
                "公司代號",
                "公司名稱",
                "當月營收",
                "上月營收",
                "去年當月營收",
                "上月比較增減(%)",
                "去年同月增減(%)",
                "當月累計營收",
                "去年累計營收",
                "前期比較增減(%)",
            ]
        else:
            data.columns = [
                "公司代號",
                "公司名稱",
                "當月營收",
                "上月營收",
                "去年當月營收",
                "上月比較增減(%)",
                "去年同月增減(%)",
                "當月累計營收",
                "去年累計營收",
                "前期比較增減(%)",
                "備註",
            ]
        data["備註"] = ""
        data = data[data["公司代號"].str[-2:] != "合計"]
        data["日期"] = ym + "10"

        if first_write:
            data.to_csv("月營收爬蟲資料.csv", encoding="utf-8", index=False)
            first_write = False
        else:
            data.to_csv(
                "月營收爬蟲資料.csv", encoding="utf-8", mode="a", index=False, header=False
            )
    except Exception as e:
        print(ym, f"爬蟲失敗：{e}")
        continue
    finally:
        time.sleep(random.randint(1, 5))
